INF6804<br>
Polytechnique Montréal<br>
TP3: Détection et suivi de multiples objets d'intérêt<br>

Auteurs:<br>
Marc-Olivier Bélanger - 1840713<br>
Pierre-Luc Chartier - 1805679

### Déclarations globales

In [52]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.optimize as sci
import torchvision.models as models
import torchvision.transforms as transforms
import urllib.request as request

In [63]:
COCO_NAMES = ['background']
label_names = request.urlopen('https://raw.githubusercontent.com/gabilodeau/INF6804/master/utils/coco-labels.txt')
for label_name in label_names.readlines():
    COCO_NAMES.append(label_name.strip().decode('UTF-8'))

IOU_THRESHOLD = 0.5
SCORE_THRESHOLD = 0.95

source_path = "/Users/mabelal/Downloads/TP3_data/frames"

model = models.detection.maskrcnn_resnet50_fpn(pretrained=True).eval()

### Détection des objets (Mask R-CNN)

In [54]:
def detect_objects(img):
    preprocess = transforms.Compose([transforms.ToTensor()])
    tensors = [preprocess(img)]
    
    predictions = model(tensors)
    boxes = []
    
    for p in predictions:
        for j, score in enumerate(p['scores']):
            if score >= SCORE_THRESHOLD:
                label = p['labels'][j]
                is_cup_or_person = (COCO_NAMES[label] == "cup") or (COCO_NAMES[label] == "person")

                if is_cup_or_person:
                    boxes.append(p['boxes'][j])
    return boxes

### Description des objets (Histogramme de couleurs)

In [55]:
def calc_color_hist(img, bbox):
    roi = (bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3])
    mask = np.zeros((img.shape[0],img.shape[1]), np.uint8)
    cv2.rectangle(mask,(roi[0],roi[1]),(roi[2],roi[3]),255,-1,8,0);
    return cv2.calcHist([img],[0],mask,[64],[0,256])


def compare_color_hist(hist1, hist2):
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)

### Association des données (Algorithme hongrois)

In [66]:
def calc_cost_matrix(prev_img, prev_bboxes, current_img, current_bboxes):
    cost_matrix = np.zeros((len(prev_bboxes), len(current_bboxes)))
    for i in range(0,len(prev_bboxes)):
        hist_prev = calc_color_hist(prev_img, prev_bboxes[i])
        for j in range(0,len(current_bboxes)):
            hist_current = calc_color_hist(current_img, current_bboxes[j])
            cost_matrix[i,j] = compare_color_hist(hist_prev, hist_current)
    return cost_matrix


def associate_data(prev_img, prev_bboxes, current_img, current_bboxes):
    cost_matrix = calc_cost_matrix(prev_img, prev_bboxes, current_img, current_bboxes)
    
    row_ind, col_ind = sci.linear_sum_assignment(cost_matrix)
    return row_ind, col_ind

### Algorithme principal

In [68]:
# keep a set of tracked objects 
# update it as needed e.g. clear the objects undetected since the last n seconds (n*fps frames)
tracked_objects = {}

# pour chaque image:
#  call détection d'objets
img = cv2.imread(os.path.join(source_path, "frame1.jpg"), cv2.IMREAD_GRAYSCALE)
boxes = detect_objects(img)

#test
img2 = cv2.imread(os.path.join(source_path, "frame2.jpg"), cv2.IMREAD_GRAYSCALE)
b2 = detect_objects(img2)

row_ind, col_ind = associate_data(img, boxes, img2, b2)
print(row_ind)
print(col_ind)
###

#  call description d'objets pour construire la matrice des coûts d'association
#  call algo hongrois et associer les objets

#  pour les objets détectés non-associés à cette étape (nouveaux objets ou réapparition)
#   update tracked objects (add to set (create their id, set their count to 0) or reset count)

#  pour les objets non-détectés à cette itération (in tracked, but not in current_detected)
#   update tracked objects (increase count, if count == threshold, remove them)

#  write bboxe coordinates of each detected and associated object (i.e. objects in tracked objects with count = 0)

[0 1 2]
[1 0 2]


### MOT17-11: Forward moving camera in a busy shopping mall

In [ ]:
# calcul du MOTA en utilisant IoU > 50%

### MOT17-05: Street scene from a moving platform

In [ ]:
# calcul du MOTA en utilisant IoU > 50%

### MOT17-13 : Filmed from a bus on a busy intersection

In [2]:
# calcul du MOTA en utilisant IoU > 50%

### Résultats sur la trame fournie

In [3]:
# call algo principal avec les trames des tasses